In [ ]:
%matplotlib inline
import datetime as dt
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from matplotlib import pyplot as plt
from alphamind.api import *
from PyFin.api import *

plt.style.use('fivethirtyeight')
engine = SqlEngine()

In [ ]:
u_name = 'zz800'
universe = Universe(u_name, [u_name])
factor_coverage = engine.fetch_factor_coverage()

In [ ]:
flitered_coverage = factor_coverage[((factor_coverage.source == 'uqer'))
                                    & (factor_coverage.universe == u_name) 
                                    & (factor_coverage.trade_date >= '2012-01-01')]
coverage_report = flitered_coverage.groupby(['factor'])['coverage'].mean()

In [ ]:
alpha_factors = coverage_report[coverage_report > 0.98].index.tolist()

#alpha_factors = ['']

alpha_factors = {
    f: DIFF(f) for f in alpha_factors
}

In [ ]:
benchmark = 906
frequency = '2w'
batch = 4
start_date = '2012-01-01'
end_date = '2017-10-24'
method = 'risk_neutral'
neutralize_risk = industry_styles + ['SIZE']

In [ ]:
data_package = fetch_data_package(engine,
                                  alpha_factors=alpha_factors,
                                  start_date=start_date,
                                  end_date=end_date,
                                  frequency=frequency,
                                  universe=universe,
                                  benchmark=benchmark,
                                  batch=batch,
                                  neutralized_risk=neutralize_risk,
                                  pre_process=[winsorize_normal],
                                  post_process=[winsorize_normal],
                                  warm_start=batch)

In [ ]:
train_x = data_package['train']['x']
train_y = data_package['train']['y']

predict_x = data_package['predict']['x']
predict_y = data_package['predict']['y']

features = data_package['x_names']

In [ ]:
def plot_model_importance(model, features):
    features = np.array(features)
    n_features = len(features)
    features_importance = model.feature_importances_
    order = features_importance.argsort().argsort()
    features = features[order >= n_features - 10]
    features_importance = features_importance[order >= n_features - 10]
    n_features = len(features)
    plt.figure(figsize=(12, 6))
    plt.barh(range(n_features), features_importance, align='center')
    plt.yticks(np.arange(n_features), features)
    plt.xlabel('Feature importance')
    plt.ylabel('Feature')

## 0. Train Score on a specific date
------------------------------------

In [ ]:
ref_date = list(train_x.keys())[-2]
sample_train_x = train_x[ref_date]
sample_train_y = train_y[ref_date].flatten()

sample_test_x = predict_x[ref_date]
sample_test_y = predict_y[ref_date].flatten()

n_estimators = 200
max_depth = 10
min_samples_split = 5
min_samples_leaf = 2

In [ ]:
model = RandomForestRegressor(n_estimators=n_estimators,
                              max_depth=max_depth,
                              min_samples_split=min_samples_split,
                              min_samples_leaf=min_samples_leaf,
                              n_jobs=-1,
                              max_features='log2')
model.fit(sample_train_x, sample_train_y)
model.score(sample_train_x, sample_train_y)

In [ ]:
model.score(sample_test_x, sample_test_y)

In [ ]:
plot_model_importance(model, features)

## 1. Train and test accuracy trend
----------

In [ ]:
dates = sorted(train_x.keys())
accuray_table = pd.DataFrame(columns=['train', 'test'])
model_df = pd.Series()

In [ ]:
for ref_date in dates:
    sample_train_x = train_x[ref_date]
    sample_train_y = train_y[ref_date].flatten()
    
    model = RandomForestRegressor(n_estimators=n_estimators,
                                  max_depth=max_depth,
                                  min_samples_split=min_samples_split,
                                  min_samples_leaf=min_samples_leaf,
                                  n_jobs=-1,
                                  max_features='log2')
    model.fit(sample_train_x, sample_train_y)
    
    train_score = model.score(sample_train_x, sample_train_y)
    accuray_table.loc[ref_date, 'train'] = train_score
    model_df.loc[ref_date] = model
    alpha_logger.info('trade_date: {0} training finished'.format(ref_date))

In [ ]:
portfolio_risk_neutralize = ['SIZE']
portfolio_industry_neutralize = True

settlement = data_package['settlement']
industry_dummies = pd.get_dummies(settlement['industry'].values)
risk_styles = settlement[portfolio_risk_neutralize].values
total_risks = settlement[neutralize_risk].values
final_res = np.zeros(len(dates))
method = 'risk_neutral'

In [ ]:
for i, ref_date in enumerate(dates):
    model = model_df[ref_date]
    sample_test_x = predict_x[ref_date]
    sample_test_y = predict_y[ref_date].flatten()
    
    cons = Constraints()
    index = settlement.trade_date == ref_date
    benchmark_w = settlement[index]['weight'].values
    realized_r = settlement[index]['dx'].values
    industry_names = settlement[index]['industry'].values
    is_tradable = settlement[index]['isOpen'].values

    cons.add_exposure(['total'], np.ones((len(is_tradable), 1)))
    cons.set_constraints('total', benchmark_w.sum(), benchmark_w.sum())

    if portfolio_industry_neutralize:
        ind_exp = industry_dummies[index]

        risk_tags = ind_exp.columns
        cons.add_exposure(risk_tags, ind_exp.values)
        benchmark_exp = benchmark_w @ ind_exp.values

        for k, name in enumerate(risk_tags):
            cons.set_constraints(name, benchmark_exp[k], benchmark_exp[k])

    if portfolio_risk_neutralize:
        risk_exp = risk_styles[index]

        risk_tags = np.array(portfolio_risk_neutralize)
        cons.add_exposure(risk_tags, risk_exp)

        benchmark_exp = benchmark_w @ risk_exp
        for k, name in enumerate(risk_tags):
            cons.set_constraints(name, benchmark_exp[k], benchmark_exp[k])

    risk_table = total_risks[index]

    y = model.predict(sample_test_x)
    test_score = model.score(sample_test_x, sample_test_y)
    accuray_table.loc[ref_date, 'test'] = test_score

    is_tradable[:] = True
    weights, analysis = er_portfolio_analysis(y,
                                              industry_names,
                                              realized_r,
                                              constraints=cons,
                                              detail_analysis=True,
                                              benchmark=benchmark_w,
                                              is_tradable=is_tradable,
                                              method=method)
    
    final_res[i] = analysis['er']['total'] / benchmark_w.sum()
    alpha_logger.info('trade_date: {0} predicting finished'.format(ref_date))

In [ ]:
last_date = advanceDateByCalendar('china.sse', dates[-1], frequency)

df = pd.Series(final_res, index=dates[1:] + [last_date])
df.sort_index(inplace=True)
df['2012-01-01':].cumsum().plot(figsize=(12, 6))
plt.title('Prod factors model {1} ({0})'.format(method, model.__class__.__name__))

In [ ]:
accuray_table.aggregate([np.mean, np.std])